# Using Naive Bayes Classifaction Model

In [ ]:
import pandas as pd
import numpy as np
import utilities
import importlib
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score,confusion_matrix,classification_report
from textblob import TextBlob

In [ ]:
importlib.reload(utilities)

### Establishing global variables

In [ ]:
PAIRS = "ModelPairs.csv"

### Loading the Data

In [ ]:
df = pd.read_csv(PAIRS, low_memory = False)
df = df.drop(columns = ["Unnamed: 0", "Unnamed: 0.1"])

print("The amount of independent pairs is {}".format(len(df[df["BinaryClass"] == 0])))
print("The amount of dependent pairs is {}".format(len(df[df["BinaryClass"] == 1])))

if (len(df[df["BinaryClass"] == 0]) == len(df[df["BinaryClass"] == 1])):
    print("Equally Split data")

### Getting the value counts of product

In [ ]:
df["req1Product"].value_counts()

### List of dataframes for each product

In [ ]:
df_core = df[df["req1Product"] == "Core"]
df_firefox = df[df["req1Product"] == "Firefox"]
df_thunderbird = df[df["req1Product"] == "Thunderbird"]
df_bugzilla = df[df["req1Product"] == "Bugzilla"]
df_seamonkey = df[df["req1Product"] == "SeaMonkey"]

### Creating the train and test sets

In [ ]:
classify_data = True
classify_type = "CountVector"
#classify_type = "one-hot-encoding"

### Choose what is test and what is train

In [ ]:
train = df_core
test = df_firefox

In [ ]:
train_x, train_y, test_x, test_y = util.train_test(train,test)

### Condense the data

In [ ]:
tfidf_transformer = TfidfTransformer()

if classify_data :
    if classify_type == "one-hot-encoding":
        train_df = df.drop(columns = ["MultiClass"])
        train_df = pd.get_dummies(train_df, drop_first=True)
        train_df.to_csv("ClassifiedPairs.csv")
    elif classify_type == "CountVector":
        count_vect = util.create_vectorizor()
        X_train_counts = count_vect.fit_transform(np.array(train_x))
        X_train_tfidf= tfidf_transformer.fit_transform(X_train_counts)
        X_test_counts = count_vect.transform(np.array(test_x))
        X_test_tfidf= tfidf_transformer.fit_transform(X_test_counts)

In [ ]:
X_train_tfidf

In [ ]:
X_test_tfidf

### Training the model

In [ ]:
clf_model = MultinomialNB().fit(X_train_tfidf,np.array(train_y).astype('int'))

### Testing the model

In [ ]:
predict_labels = clf_model.predict(X_test_tfidf)
actualLabels = np.array(test_y).astype('int')

In [ ]:
clf_test_score = clf_model.score(X_test_tfidf,actualLabels)
print("\n"+100*"-")
print(" Classifier Test Score : "+str(clf_test_score))


f1 = round(f1_score(actualLabels, predict_labels,average='macro'),2)
precision = round(precision_score(actualLabels, predict_labels,average='macro'),2)
recall = round(recall_score(actualLabels, predict_labels,average='macro'),2)